In [2]:
import sys
import os
import json
import datetime

## 아마존 EC2에 올라 갔을때 모듈 위치를 제대로 못잡을수 있으니 아래와 같은 조치를 함.
import numpy as np
import pandas as pd
import pymysql
import statsmodels.api as sm
import statsmodels.formula.api as smf
conn =pymysql.connect(
    host="fermi-master.c4kp2nxu0eer.ap-northeast-2.rds.amazonaws.com",
    user="admin",
    password="fermi1234",
    db="energy_data",
    charset='utf8'
)
curs=conn.cursor()
sql="""
select smp_price.date, smp_price.total_price as smp_price, oil_price.wti,elec_supply.supply,price_index.korea_producer, price_index.us_producer from smp_price
    inner join oil_price
		on smp_price.date=oil_price.date
	inner join elec_supply
		on smp_price.date=elec_supply.date
	inner join price_index
		on smp_price.date=price_index.date
        """
curs.execute(sql)
array=np.array(curs.fetchall())
data=pd.DataFrame(data=array[:,1:6],index=array[:,0],columns=["smp_price","oil_price","elec_supply","korea_producer","us_producer"])
data.smp_price=data.smp_price*data.korea_producer
data.oil_price=data.oil_price*data.us_producer
data=data.astype(float)
type(data.index[0])

/Users/youngji/.local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


datetime.date

In [19]:
def find_fields(tables):
    sql="SELECT * FROM information_schema.columns WHERE Table_Name=%s"
    for i in range(1,len(tables)):
        sql +=" OR Table_Name=%s"
    curs=conn.cursor()
    curs.execute(sql,tables)
    fields=[]
    fields_result=curs.fetchall()
    for i in fields_result:
        fields.append(i[3])
    return fields

def fetch_forecast(scenes):
    fields=find_fields(['elec_forecast'])
    sql="select * from energy_data.elec_forecast where date"
    curs=conn.cursor()
    curs.execute(sql)
    result=pd.DataFrame(data=np.array(curs.fetchall()),columns=fields)
    result.index=result.date
    final_result=result.loc[:,scenes]
    final_result=final_result.astype(float)
    return(final_result)

def make_scenario(model,forecast):
    pred=model.predict(forecast)
    output=pd.DataFrame({'smp_price':pred})
    output=output.astype(int)
    return(output)

model=smf.ols(formula = 'smp_price~oil_price+elec_supply', data = data).fit()
forecast=fetch_forecast(["WEO_450","supply7"])
forecast=forecast.rename(columns={"WEO_450":"oil_price","supply7":"elec_supply"})
pred=make_scenario(model,forecast)
pred


,smp_price
date,
2018-04-01,86
2018-05-01,86
2018-06-01,85
2018-07-01,85
2018-08-01,85
2018-09-01,84
2018-10-01,84
2018-11-01,84
2018-12-01,83


In [19]:
params = { 'size': 99, 'type': 1.2,'average_time':3.4}
params['size']

99

In [20]:
pred['rec_price']=100;pred['size']=params['size'];pred['type']=params['type'];pred['average_time']=params['average_time']
pred['smp_revenue']=pred['smp_price']*30*pred['type']
pred['rec_revenue']=pred['rec_price']*30*pred['type']


,smp_price,rec_price,size,type,average_time,smp_revenue,rec_revenue
date,,,,,,,
2018-04-01,86,100,99,1.2,3.4,3096.0,3600.0
2018-05-01,86,100,99,1.2,3.4,3096.0,3600.0
2018-06-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-07-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-08-01,85,100,99,1.2,3.4,3060.0,3600.0
2018-09-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-10-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-11-01,84,100,99,1.2,3.4,3024.0,3600.0
2018-12-01,83,100,99,1.2,3.4,2988.0,3600.0


In [21]:
pred['total_revenue']=pred['smp_revenue']+pred['rec_revenue']

In [22]:
pred

,smp_price,rec_price,size,type,average_time,smp_revenue,rec_revenue,total_revenue
date,,,,,,,,
2018-04-01,86,100,99,1.2,3.4,3096.0,3600.0,6696.0
2018-05-01,86,100,99,1.2,3.4,3096.0,3600.0,6696.0
2018-06-01,85,100,99,1.2,3.4,3060.0,3600.0,6660.0
2018-07-01,85,100,99,1.2,3.4,3060.0,3600.0,6660.0
2018-08-01,85,100,99,1.2,3.4,3060.0,3600.0,6660.0
2018-09-01,84,100,99,1.2,3.4,3024.0,3600.0,6624.0
2018-10-01,84,100,99,1.2,3.4,3024.0,3600.0,6624.0
2018-11-01,84,100,99,1.2,3.4,3024.0,3600.0,6624.0
2018-12-01,83,100,99,1.2,3.4,2988.0,3600.0,6588.0
